# Data Wrangling

of Project 'Demography behind household sizes in Switzerland and its change in time'

In [529]:
%matplotlib inline
import os
import pandas as pd
import json
import folium
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [530]:
data_folder = './data/'
wage_folder = data_folder+'Gross_monthly_wage_GMW/'

## 1) Load and Prepare Data

In [531]:
#Load raw data and load entire excel sheets
hh_sizes = pd.read_excel(data_folder + 'HH_sizes_commune.xlsx', skiprows=3, skipfooter=10, sheet_name = None, usecols=list(range(9)))
hh_size_agglo = pd.read_excel(data_folder + 'HH_aglo_rural .xlsx', skiprows=7, skipfooter=10, sheet_name = None)
incomes = pd.read_csv(data_folder + 'Incomes.csv')
cantonal_parties = pd.read_excel(data_folder + 'Kantonale Parlamentswahlen.xls', skiprows=3, skipfooter=10, sheet_name = None)
foreign_population = pd.read_excel(data_folder + 'population_regions.xlsx', skiprows=4, skipfooter=10, sheet_name = None)
wage_swiss_foreign_ch = pd.read_excel(wage_folder + 'GMW_switzerland.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
wage_swiss_foreign_lake_ge = pd.read_excel(wage_folder + 'GMW_lake_geneva.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
wage_swiss_foreign_mittelland = pd.read_excel(wage_folder + 'GMW_mittelland.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
wage_swiss_foreign_north = pd.read_excel(wage_folder + 'GMW_north.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
wage_swiss_foreign_zh = pd.read_excel(wage_folder + 'GMW_zurich.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
wage_swiss_foreign_east = pd.read_excel(wage_folder + 'GMW_east.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
wage_swiss_foreign_central = pd.read_excel(wage_folder + 'GMW_central.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
wage_swiss_foreign_ti = pd.read_excel(wage_folder + 'GMW_ticino.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
rental_11_13_rooms = pd.read_excel(data_folder + 'rental_prices_age_11-13.xls', skiprows=8, skipfooter=10)
rental_13_16_all = pd.read_excel(data_folder + 'rental_prices_age.xlsx', skiprows=5, skipfooter=11,sheet_name = None)
rental_foreign_rooms = pd.read_excel(data_folder + 'rental_prices_nationality.xlsx', skiprows=4, skipfooter=11,sheet_name = None)
rental_rooms_cities_11_13 = pd.read_excel(data_folder + 'rental_prices_room_size_11-13.xls', skiprows=8, skipfooter=12)
rental_rooms_cities_12_14 = pd.read_excel(data_folder + 'rental_prices_room_size_12-14.xls', skiprows=7, skipfooter=10)
rental_sqm_demographics = pd.read_excel(data_folder + 'rental_prices_size.xlsx', skiprows=3, skipfooter=21, sheet_name = None)


In [532]:
#definitions to apply arguments to all sheets of the excel
#rename headers
def rename_header(df, header):
    for y in df.keys():
        df[y].columns = header
    return df

def drop_NA(df):
    for y in df.keys():
        df[y] = df[y].replace(r'^\s*$',np.nan,regex=True)
        df[y] = df[y].dropna(axis= 0,how = 'all')
        df[y] = df[y].dropna(axis= 1,how = 'all')
    return df

def select_X_rows(df,num_rows):
    for y in df.keys():
        df[y] = df[y].iloc[:num_rows]
    return df

def rename_columns(df, new_names):
    for y in df.keys():
        df[y] = df[y].rename(index=str, columns=new_names)
    return df

def drop_index(df, ind, rows = True):
    if rows:
        for y in df.keys():
            df[y] = df[y].drop(ind).reset_index().drop('index', 1)
    else:
        for y in df.keys():
            df[y] = df[y].drop(df[y].columns[ind], axis=1)
    return df

def clean_wage_region_data(df):
    header= ["Residents","total_median","quartile_from","quartile_to",
            "women_median","women_quartile_from","woman_quartile_to",
            "men_median","men_quartile_from","man_quartile_to"]
    df = drop_NA(df)
    df = select_X_rows(df,8)
    df = rename_header(df, header)
    return df

def clean_rental_csv(df):
    drop_cols = [2,4,6,8,10,12,14]
    df = df.drop(df.columns[drop_cols], axis=1)
    header = ["Region","average_rent","1 room","2 rooms",
                "3 rooms","4 rooms","5 rooms",
                "6+ rooms"]
    df.columns = header
    df = df.dropna(axis= 0,how = 'all')
    return df

In [533]:
#for household size: 
col_header = ['Commune', 'Total', '1-person households','2-person households', 
               '3-person households', '4-person households','5-person households', 
               '6-person or larger households', 'not plausible hh']
hh_sizes = rename_header(hh_sizes, col_header)
hh_sizes['2017'].head()

,Commune,Total,1-person households,2-person households,3-person households,4-person households,5-person households,6-person or larger households,not plausible hh
0,Switzerland,8317056,1320230,2434588,1477575,1892056,792445,400162,< 0.5
1,0001 Aeugst am Albis,1931,233,566,351,516,170,95,< 0.5
2,0002 Affoltern am Albis,11741,1771,3498,1995,2628,1205,644,< 0.5
3,0003 Bonstetten,5491,646,1606,960,1620,500,159,0
4,0004 Hausen am Albis,3641,432,1084,609,980,365,171,0


In [534]:
cantonal_parties = drop_NA(cantonal_parties)
cantonal_parties = select_X_rows(cantonal_parties,26)
rename_cols = {"Unnamed: 0": "Canton"}
cantonal_parties = rename_columns(cantonal_parties, rename_cols)
cantonal_parties['aktuell (2015-2018)'].head()

,Canton,Wahljahr 2),Wahlbeteiligung,FDP 6),CVP 7),SP,SVP,LPS 6),EVP,CSP,...,PSA,GPS,FGA 8),Sol.,SD,EDU,Lega,MCR,Übrige 11),Total
2,Zürich,2015.0,32.652497,17.327820,4.878712,19.716444,30.023215,NaN,4.271767,NaN,...,NaN,7.218776,2.977664,NaN,NaN,2.662277,NaN,NaN,0.669707,100
3,Bern,2018.0,30.516313,11.717910,0.671415,22.328775,26.760869,NaN,6.172897,NaN,...,0.681873,10.104467,0.495841,NaN,0.179432,3.710624,NaN,NaN,0.912781,100
4,Luzern,2015.0,38.741267,21.039516,30.862513,11.848915,24.115646,NaN,0.199143,NaN,...,NaN,6.700011,NaN,NaN,NaN,NaN,NaN,NaN,0.036129,100
5,Uri 1),2016.0,61.989056,26.856698,31.298814,12.984966,24.053191,NaN,NaN,NaN,...,NaN,2.378061,NaN,NaN,NaN,NaN,NaN,NaN,2.428270,100
6,Schwyz,2016.0,37.747088,21.628996,27.167707,12.925363,33.115090,NaN,0.304428,NaN,...,NaN,0.800215,NaN,NaN,NaN,NaN,NaN,NaN,1.513578,100


In [535]:
foreign_population = drop_NA(foreign_population)
rename_cols= {"Unnamed: 0": "Region",
             "Unnamed: 1":"Population",
             "Urban core":"Urban core area",
             "Area influenced":"influenced by urban cores",
             "Area beyond":"influenced beyond urban cores"}
foreign_population = rename_columns(foreign_population, rename_cols)
drop_indices = ['0','1']
foreign_population = drop_index(foreign_population, drop_indices)
foreign_population['2017'].head()

,Region,Population,0–19,20–64,65 and over,Male,Female,Swiss,Foreigner,Single,Married,Widowed,Divorced,Unmarried,In a registered partnership,Partnership dissolved,Urban core area,influenced by urban cores,influenced beyond urban cores
0,Total,8484130.0,1700494.0,5233271.0,1550365.0,4206434.0,4277696.0,6357738.0,2126392.0,3755372.0,3592368.0,406117.0,711025.0,588.0,16435.0,2001.0,5339043,1852223,1292864
1,Lake Geneva region,1629841.0,346875.0,1005903.0,277063.0,798780.0,831061.0,1086874.0,542967.0,747544.0,661155.0,73063.0,143847.0,163.0,3558.0,458.0,1129428,344955,155458
2,Vaud,793129.0,174446.0,488905.0,129778.0,389504.0,403625.0,527117.0,266012.0,368315.0,319792.0,34382.0,68554.0,57.0,1786.0,216.0,491022,219482,82625
3,Valais,341463.0,67825.0,207639.0,65999.0,169162.0,172301.0,263082.0,78381.0,145510.0,149095.0,18326.0,28051.0,35.0,378.0,52.0,187341,81289,72833
4,Geneva,495249.0,104604.0,309359.0,81286.0,240114.0,255135.0,296675.0,198574.0,233719.0,192268.0,20355.0,47242.0,71.0,1394.0,190.0,451065,44184,0


In [536]:
wage_swiss_foreign_ch = clean_wage_region_data(wage_swiss_foreign_ch)
wage_swiss_foreign_lake_ge = clean_wage_region_data(wage_swiss_foreign_lake_ge)
wage_swiss_foreign_mittelland = clean_wage_region_data(wage_swiss_foreign_mittelland)
wage_swiss_foreign_north = clean_wage_region_data(wage_swiss_foreign_north)
wage_swiss_foreign_zh = clean_wage_region_data(wage_swiss_foreign_zh)
wage_swiss_foreign_east = clean_wage_region_data(wage_swiss_foreign_east)
wage_swiss_foreign_central = clean_wage_region_data(wage_swiss_foreign_central)
wage_swiss_foreign_ti = clean_wage_region_data(wage_swiss_foreign_ti)

**Note :** The wage level with "No management function" was chosen.

In [537]:
wage_swiss_foreign_zh['2016']

,Residents,total_median,quartile_from,quartile_to,women_median,women_quartile_from,woman_quartile_to,men_median,men_quartile_from,man_quartile_to
1,TOTAL,5921,4772.0,7379.0,5374,4459,6864,6282.0,5166.0,7794.0
3,Swiss,6136,4951.0,7559.0,5620,4633,7000,6584.0,5398.0,8113.0
5,Foreigners,5534,4498.0,6955.0,4826,4135,6468,5849.0,4860.0,7175.0
7,Short-term permit (L),5471,4444.0,6734.0,[4 473],[3 650],[6 588],5568.0,4882.0,6746.0
8,Temporary residence permit (B),5284,4294.0,7059.0,4862,4034,6795,5514.0,4509.0,7221.0
9,Permanent residence permit (C),5595,4584.0,6872.0,4782,4170,6189,5967.0,5060.0,7136.0
10,Cross-border worker (G),5860,4808.0,7263.0,5035,4385,6469,6157.0,5178.0,7579.0
11,Other,4234,3784.0,5709.0,4329,3796,5773,4162.0,3774.0,5590.0


In [538]:
drop_cols = [2,4,6,8,10,12,14,16]
rental_11_13_rooms = rental_11_13_rooms.drop(rental_11_13_rooms.columns[drop_cols], axis=1)
header = ["Region","average_rent","<25","25-64",
            "65+","<25 and 25-64","<25 and 65+",
            "25-64 and 65+","<25, 25-64, and 65+"]
rental_11_13_rooms.columns = header
rental_11_13_rooms = rental_11_13_rooms.dropna(axis= 0,how = 'all')
drop_rows = ["1 Zimmer","2 Zimmer","3 Zimmer","4 Zimmer", "5 Zimmer", "6 Zimmer und mehr"]

for value in drop_rows:
    rental_11_13_rooms = rental_11_13_rooms.drop(rental_11_13_rooms.index[rental_11_13_rooms["Region"] == value])
rental_11_13_rooms.head()

,Region,average_rent,<25,25-64,65+,<25 and 25-64,<25 and 65+,25-64 and 65+,"<25, 25-64, and 65+"
0,Schweiz,1317.0,1089,1275.0,1134,1527,1293,1318.0,1570
8,Genferseeregion,1309.0,1066,1233.0,1076,1575,1341,1277.0,1612
16,Espace Mittelland,1154.0,983,1114.0,1018,1334,***** 1'178,1173.0,1406
24,Nordwestschweiz,1323.0,1127,1281.0,1171,1519,***** 1'318,1322.0,1547
32,Zürich,1526.0,1263,1492.0,1325,1743,***** 1'372,1528.0,1751


**Legend :** Average rental cost in which specific demographics reside. For instance the column "<25, 25-64, and 65+" means that in this household, resides at least a person that is less than 25 years, a person between 25-64 years, and also a person who is 65 and older. 

In [539]:
drop_cols = [2,4,6,8,10,12,14,16]
rental_13_16_all = drop_index(rental_13_16_all, drop_cols, rows = False)
rental_13_16_all = rename_header(rental_13_16_all, header)
rental_13_16_all['2016']

,Region,average_rent,<25,25-64,65+,<25 and 25-64,<25 and 65+,25-64 and 65+,"<25, 25-64, and 65+"
0,Suisse,1322,1073,1278,1124,1538,1368,1316,1574
1,Genferseeregion,1316,1007,1239,1062,1588,1220,1281,1607
2,Espace Mittelland,1145,972,1108,989,1332,1320,1140,1400
3,Nordwestschweiz,1328,1122,1277,1185,1522,1263,1328,1477
4,Zürich,1553,1267,1517,1312,1776,1641,1550,1759
5,Ostschweiz,1189,1041,1148,1069,1351,1581,1206,1408
6,Zentralschweiz,1446,1169,1417,1208,1678,1338,1428,1540
7,Tessin,1150,941,1086,1047,1345,X,1182,1513


In [540]:
drop_cols = [2,4,6,8]
rental_foreign = drop_index(rental_foreign_rooms, drop_cols, rows = False)
header = ["Region","average rent all","household with only Swiss",
            "hh with min 1 Swiss and 1 foreigner","household with only foreigners"]
rental_foreign = rename_header(rental_foreign, header)

drop_rows = ["1 Zimmer","2 Zimmer","3 Zimmer","4 Zimmer", "5 Zimmer", "6 Zimmer und mehr"]
drop_ind = []
for i, value in enumerate(rental_foreign['2016']["Region"]):
    for item in drop_rows:
        if value == item:
            drop_ind.append(i)

rental_foreign = drop_index(rental_foreign, drop_ind)
rental_foreign['2016']

,Region,average rent all,household with only Swiss,hh with min 1 Swiss and 1 foreigner,household with only foreigners
0,Schweiz,1322,1300,1504,1305
1,Genferseeregion,1316,1258,1505,1343
2,Espace Mittelland,1145,1149,1302,1073
3,Nordwestschweiz,1328,1317,1489,1293
4,Zürich,1553,1517,1707,1564
5,Ostschweiz,1189,1207,1348,1103
6,Zentralschweiz,1446,1407,1650,1491
7,Tessin,1150,1156,1288,1097


In [541]:
rental_rooms_cities_11_13 = clean_rental_csv(rental_rooms_cities_11_13)
rental_rooms_cities_11_13

,Region,average_rent,1 room,2 rooms,3 rooms,4 rooms,5 rooms,6+ rooms
0,Schweiz,1317.0,726,1027,1252,1517.0,1853,2397
2,Zürich,1490.0,887,1274,1491,1834.0,2301,3132
3,Genève,1381.0,865,1124,1374,1676.0,2069,3154
4,Basel,1231.0,682,975,1247,1603.0,1985,2987
5,Lausanne,1187.0,722,985,1229,1569.0,1945,2815
6,Bern,1255.0,704,1041,1263,1560.0,1925,2516
7,Winterthur,1555.0,X,***** 1'101,***** 1'403,1766.0,***** 1'959,***** 2'779
8,Luzern,1358.0,748,1088,1289,1549.0,2019,2656
9,St.Gallen,1194.0,603,931,1120,1327.0,1595,2154
10,Lugano,1224.0,697,1013,1234,1484.0,1897,2387


In [542]:
rental_rooms_cities_12_14 = clean_rental_csv(rental_rooms_cities_12_14)
rental_rooms_cities_12_14

,Region,average_rent,1 room,2 rooms,3 rooms,4 rooms,5 rooms,6+ rooms
1,Schweiz,1332.0,735.0,1039.0,1265.0,1532.0,1866.0,2407
3,Zürich,1514.0,911.0,1295.0,1501.0,1839.0,2296.0,3214
4,Genève,1392.0,890.0,1133.0,1380.0,1686.0,2048.0,2941
5,Basel,1246.0,681.0,989.0,1259.0,1638.0,2037.0,2900
6,Lausanne,1213.0,746.0,999.0,1256.0,1588.0,1988.0,2794
7,Bern,1280.0,708.0,1041.0,1279.0,1545.0,1985.0,2605
8,Winterthur,1397.0,826.0,1148.0,1303.0,1596.0,1967.0,2489
9,Luzern,1375.0,755.0,1111.0,1296.0,1557.0,2091.0,2752
10,St.Gallen,1190.0,607.0,922.0,1131.0,1313.0,1576.0,2161
11,Lugano,1237.0,713.0,1025.0,1252.0,1497.0,1912.0,2403


In [543]:
# das chönnt me wahrschinli au besser cleanä
drop_ind = [2,4,6,8]
rental_sqm_demographics = drop_index(rental_sqm_demographics, drop_ind, rows = False)
rental_sqm_demographics = drop_NA(rental_sqm_demographics)
rental_sqm_demographics['2016']

,Unnamed: 0,Unnamed: 1,ohne Migrations-hintergrund,mit Migrations-hintergrund 4),Gemischt mit/ohne Migrationshintergrund 4)
2,Total,15.8,15.1,16.7,16.5
3,Einpersonenhaushalte,16.0,15.3,17.2,*
4,Paare ohne Kinder,15.8,15,16.8,16.7
5,Paare mit Kind(ern),15.7,14.8,16.3,16
6,Einelternhaushalte,14.9,14.2,15.6,*
7,Nichtfamilienhaushalte mit mehreren Personen,17.1,16.3,17.9,18.1
8,Mehrfamilienhaushalte,14.8,13.8,14.8,15.5
10,Genferseeregion,NaN,NaN,NaN,NaN
11,Einpersonenhaushalte,17.1,15.7,18.7,*
12,Paare ohne Kinder,16.7,15,18.0,17.6


In [544]:
# Kei Ahnig wie me das chan cleanä 
hh_size_agglo['2017']

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Schweiz,NaN,1,35.460703,NaN
1,NaN,NaN,2,32.695895,2.233919
2,NaN,NaN,3+,31.843402,NaN
3,NaN,Regionalisierung 3),NaN,NaN,NaN
4,Zürich,NaN,1,44.936295,NaN
5,NaN,Kernstadt,2,29.917171,1.998171
6,NaN,NaN,3+,25.146534,NaN
7,NaN,NaN,1,32.217496,NaN
8,NaN,Agglomerationsgürtel,2,34.449090,2.288234
9,NaN,NaN,3+,33.333414,NaN
